In [1]:
import pulp
import pandas as pd


In [2]:
T = ['Arsenal',
     'AstonVilla',
     'Bournemouth',
     'Brentford',
     'Brighton',
     'Chelsea',
     'CrystalPalace',
     'Everton',
     'Fulham',
     'LeedsUnited',
     'LeicesterCity',
     'Liverpool',
     'ManchesterCity',
     'ManchesterUnited',
     'NewcastleUnited',
     'NottinghamForest',
     'Southampton',
     'TottenhamHotspur',
     'WestHamUnited',
     'WolverhamptonWanderers'
     ]

Liverpool = ['Liverpool', 'Everton']
Manchester = ['ManchesterCity', 'ManchesterUnited']

In [3]:
W = range(1, 2*len(T) - 1)
W2 = range(1, 2*len(T) - 2)
W_first_half = range(1, len(T))
W_second_half = range(len(T), 2*len(T) - 1)


In [4]:
# Decision variable indicating team h plays against team a in week w
x = pulp.LpVariable.dicts('x', (W, T, T), cat=pulp.LpBinary)
y = pulp.LpVariable.dicts('y', (W2, T), cat=pulp.LpBinary)


In [5]:
# Team cannot play itself
for w in W:

    for h in T:

        for a in T:

            if h == a:

                del x[w][h][a]

schedule = pulp.LpProblem("Schedule", pulp.LpMinimize)


In [6]:
schedule += pulp.lpSum(y[w][i] for w in W2 for i in T)


In [7]:
# Each team plays exactly one match each week
for w in W:

    for i in T:

        schedule += pulp.lpSum(x[w][i][a] for a in T if a != i) + \
            pulp.lpSum(x[w][h][i] for h in T if h != i) == 1


In [8]:
# Teams play each other twice during the season, with one home match and one away match
for i in T:
    for j in T:
        if j != i:
            # team i plays one match in the first half against team j
            schedule += pulp.lpSum(x[w][i][j] for w in W_first_half) + \
                pulp.lpSum(x[w][j][i] for w in W_first_half) == 1
            # team i plays one match in the second half against team j
            schedule += pulp.lpSum(x[w][i][j] for w in W_second_half) + \
                pulp.lpSum(x[w][j][i] for w in W_second_half) == 1
            # if team i played at home against team j in the first half,
            # they will play away against team j in the second half, and vice versa
            schedule += pulp.lpSum(x[w][i][j] for w in W_first_half) == pulp.lpSum(
                x[w][j][i] for w in W_second_half)
            schedule += pulp.lpSum(x[w][j][i] for w in W_first_half) == pulp.lpSum(
                x[w][i][j] for w in W_second_half)


In [9]:
# First two and last two games cannot be consecutive away or home matches
for i in T:
    schedule += pulp.lpSum(x[W[0]][i][j] for j in T if i != j) + \
        pulp.lpSum(x[W[1]][i][j] for j in T if i != j) == 1
    schedule += pulp.lpSum(x[W[-1]][i][j] for j in T if i != j) + \
        pulp.lpSum(x[W[-2]][i][j] for j in T if i != j) == 1


In [10]:
# If a team is home during boxing day (week 17) it will be away during new year's day (week 18)
for i in T:
    schedule += pulp.lpSum(x[W[16]][i][j] for j in T if i != j) + \
        pulp.lpSum(x[W[17]][i][j] for j in T if i != j) == 1


In [11]:
# Sequencing rule
for i in T:
    for w in range(1, 2*len(T) - 5):
        schedule += pulp.lpSum(x[w][i][j] + x[w + 1][i][j] + x[w + 2][i][j] + x[w + 3][i][j] + x[w + 4][i][j]
                               for j in T if i != j) <= 3
        schedule += pulp.lpSum(x[w][i][j] + x[w + 1][i][j] + x[w + 2][i][j] + x[w + 3][i][j] + x[w + 4][i][j]
                               for j in T if i != j) >= 2


In [12]:
# Teams from the same city can't play both home in the same week
for w in W:
    schedule += pulp.lpSum(x[w][h][a] for a in T for h in Liverpool if h != a) == len(Liverpool) // 2

for w in W:
    schedule += pulp.lpSum(x[w][h][a] for a in T for h in Manchester if h != a) == len(Manchester) // 2

In [13]:
# Minimize the number of consecutive home and away games
for i in T:
    for w in W2:
        schedule += pulp.lpSum(x[w][i][j] + x[w + 1][i][j] for j in T if i != j) - y[w][i] == 1

In [14]:
schedule.writeLP("file.lp");
